In [1]:
from tqdm import tqdm
from hitbtc import Hitbtc
from urllib.request import urlopen
from bittrex.bittrex import Bittrex
from bitfinex import Bitfinex
import krakenex

exchanges = {}
btc = 0.01

## Read from HitBTC

In [2]:
hitbtcClient = Hitbtc(None, None)

tickers = hitbtcClient.get_tickers()
currencies = hitbtcClient.get_currencies()
symbols = hitbtcClient.get_symbol('')

hitbtc = {}

pbar = tqdm(range(len(tickers)))
pbar.set_description('Processing')

for j in pbar:
    tick = tickers[j]
    
    for d in currencies:
        if d['id'] == tick['symbol'][0:-3]:
            currency = d
            
    for d in symbols:
        if d['id'] == tick['symbol']:
            symbol = d
    
    try:
        ask = float(tick['ask'])
    except:
        ask = 0.0
    try:
        bid = float(tick['bid'])
    except:
        bid = 0.0
        
    hitbtc[tick['symbol']]={'ask':ask,'bid':bid, 'base':tick['symbol'][-3:],
                            'payinEnabled':currency['payinEnabled'],
                            'payoutEnabled':currency['payoutEnabled'],
                            'takerFee':symbol['takeLiquidityRate'],
                           'withdrawalFee':None}
    # TODO - withdraw fee not yet implemented on API
        
exchanges['hitbtc'] = hitbtc

Processing: 100%|███████████████████████████████████████████████████████████████████████████████| 425/425 [00:00<00:00, 8012.02it/s]


## Read from Bittrex

In [3]:
bittrexClient = Bittrex(None, None)  # or defaulting to v1.1 as Bittrex(None, None)
symbols = bittrexClient.get_market_summaries()['result']
currencies = bittrexClient.get_currencies()['result']

bittrex = {}

pbar = tqdm(range(len(symbols)))
pbar.set_description('Processing')

for j in pbar:
    symbol = symbols[j]
    
    names = symbol['MarketName'].split('-')
    
    # Get withdrawal fee
    for d in currencies:
        if d['Currency'] == names[1]:
            currency = d

    try:
        ask = float(symbol['Ask'])
    except:
        ask = 0.0
    try:
        bid = float(symbol['Bid'])
    except:
        bid = 0.0
    
    # Casos a parte
    if names[1] == 'BCC':
        names[1] = 'BCH'
        
    bittrex[names[1]+names[0]] = {'ask':ask,'bid':bid, 'base':names[0],
                                  'payinEnabled':True,
                                  'payoutEnabled':True,
                                 'takerFee':'0.0025',
                                 'withdrawalFee': currency['TxFee']}
    
exchanges['bittrex'] = bittrex

Processing: 100%|███████████████████████████████████████████████████████████████████████████████| 266/266 [00:00<00:00, 6563.28it/s]


## Read from Binance

In [4]:
from binance.client import Client
binanceClient = Client("", "")

symbols = binanceClient.get_orderbook_tickers()

products = binanceClient.get_products()['data']

binance = {}

pbar = tqdm(range(len(symbols)))
pbar.set_description('Processing')

for j in pbar:
    symbol = symbols[j]
    
    # TODO - Get withdrawal fee - Not working
    for d in products:
        if d['symbol'] == symbol['symbol']:
            currency = d
    
    try:
        ask = float(symbol['askPrice'])
    except:
        ask = 0.0
    try:
        bid = float(symbol['bidPrice'])
    except:
        bid = 0.0
        
    if 'BCC' in symbol['symbol']:
            symbol['symbol'] = symbol['symbol'].replace('BCC', 'BCH')
        
    binance[symbol['symbol']] = {'ask':ask,'bid':bid, 'base':symbol['symbol'][-3:],
                                 'payinEnabled':True,
                                 'payoutEnabled':True,
                                 'takerFee':'0.001',
                                 'withdrawalFee': currency['withdrawFee']}
        
exchanges['binance'] = binance

Processing: 100%|███████████████████████████████████████████████████████████████████████████████| 231/231 [00:00<00:00, 7568.88it/s]


## All possible transactions

In [5]:
trades = list()

for exchangeVenda in exchanges:
    for coin in exchanges[exchangeVenda]:
        for exchangeCompra in exchanges:
            if exchangeVenda != exchangeCompra:
                try:                   
                    if exchanges[exchangeCompra][coin]['base'] == exchanges[exchangeVenda][coin]['base']:                      
                        base = exchanges[exchangeVenda][coin]['base']
                        
                        priceSell = exchanges[exchangeVenda][coin]['ask'] # Price exchange is selling
                        priceBuy = exchanges[exchangeCompra][coin]['bid'] # Price exchange is buying
                        sellFee = float(exchanges[exchangeVenda][coin]['takerFee']) # Fee for the selling exchange
                        buyFee = float(exchanges[exchangeCompra][coin]['takerFee']) # Fee for the buying exchange
                        
                        buy = btc/priceSell * (1-sellFee)       # Quantity of coin bought
                        transfer = buy*(0.99)                   # Quantity of coin transfered after withdraw fee
                        sell = transfer * priceBuy * (1-buyFee) # Quantity of coin sold
                        transferBTC = sell*(0.99)               # Quantity of coin transfered back after withdraw fee

                        gain = (transferBTC-btc)/btc            # Final gain

                        if gain > 0.00 and \
                            exchanges[exchangeVenda][coin]['payoutEnabled'] == True and \
                            exchanges[exchangeCompra][coin]['payinEnabled'] == True:
                            trades.append({'coin':coin.replace(base, ''),
                                           'sell':priceSell,
                                           'buy':priceBuy,
                                           'gain':int(gain*10000)/100,
                                           'from':exchangeVenda,
                                           'to':exchangeCompra,
                                           'base':base})
                except:
                    ibis = 1
    break
    
trades = sorted(trades, key=lambda k: k['gain'],reverse=True) 

for i in trades:
    print(i)

{'coin': 'MAID', 'sell': 6.257e-05, 'buy': 6.985e-05, 'gain': 9.03, 'from': 'hitbtc', 'to': 'bittrex', 'base': 'BTC'}


## Tests with coins

In [ ]:
# HitBTC
orderbook = hitbtcClient.get_ticker('AEONBTC')
venda = float(orderbook['ask'])
print('Preço de venda:', venda)

# Bittrex
orderbook = bittrexClient.get_ticker('BTC-AEON')['result']
compra = float(orderbook['Bid'])
print('Preço de compra:', compra)

# Quantidade Inicial
btc = 0.1
buy = btc/venda * (1-0.25/100)
print(buy)
transfer = buy - 0.01
sell = transfer * compra * (1-0.1/100)
transferBTC = sell - 0.0009

gain = (transferBTC-btc)/btc

print('Ganho:', int(gain * 10000)/100)

In [ ]:
# Bittrex
orderbook = bittrexClient.get_ticker('BTC-DCT')['result']
venda = float(orderbook['Ask'])
print('Preço de venda:', venda)

# HitBTC
orderbook = hitbtcClient.get_ticker('DCTBTC')
compra = float(orderbook['bid'])
print('Preço de compra:', compra)

# Quantidade Inicial
btc = 0.01
buy = btc/venda * (1-0.25/100)
transfer = buy - 0.1
sell = transfer * compra * (1-0.1/100)
transferBTC = sell - 0.0009

gain = (transferBTC-btc)/btc

print('Ganho:', int(gain * 10000)/100)

In [ ]:
# HitBTC
orderbook = hitbtcClient.get_ticker('EVXBTC')
venda = float(orderbook['ask'])
print('Preço de venda:', venda)

# Binance
orderbook = binanceClient.get_order_book(symbol='EVXBTC')
compra = float(orderbook['bids'][0][0])
print('Preço de venda:', compra)

# Quantidade Inicial
btc = 0.01
buy = btc/venda * (1-0.25/100)
transfer = buy - 5.4
sell = transfer * compra * (1-0.1/100)
transferBTC = sell - 0.0009

gain = (transferBTC-btc)/btc

print('Ganho:', int(gain * 10000)/100)

In [ ]:
# HitBTC
orderbook = hitbtcClient.get_orderbook('TRSTBTC')
compra = float(orderbook['bid'][0]['price'])
print('Preço de compra:', compra)

# Bittrex
orderbook = bittrexClient.get_orderbook('BTC-TRST')['result']
venda = float(orderbook['sell'][0]['Rate'])
print('Preço de venda:', venda)

# Quantidade Inicial
btc = 0.01
buy = btc/venda * (1-0.25/100)
transfer = buy - 0.01
sell = transfer * compra * (1-0.1/100)
transferBTC = sell - 0.0009

gain = (transferBTC-btc)/btc

print('Ganho:', int(gain * 10000)/100)

In [ ]:
# HitBTC
orderbook = client.get_orderbook('DGBBTC')
compra = float(orderbook['bid'][0]['price'])
print('Preço de compra:', compra)

# Bittrex
orderbook = my_bittrex.get_orderbook('BTC-DGB')['result']
venda = float(orderbook['sell'][0]['Rate'])
print('Preço de venda:', venda)

# Quantidade Inicial
btc = 0.01
buyXDN = btc/venda * (1-0.1/100)
transferXDN = buyXDN - 0.01
sellXDN = transferXDN * compra * (1-0.25/100)
transferBTC = sellXDN - 0.001

gain = (transferBTC-btc)/btc

print(gain * 100)

In [ ]:
# HitBTC
orderbook = client.get_orderbook('XDNBTC')
venda = orderbook['ask'][0]['price']
venda = float(venda)

# Bittrex
response = urlopen('https://bittrex.com/api/v1.1/public/getorderbook?market=BTC-XDN&type=both')
string = response.read().decode('utf-8')
json_obj = json.loads(string)
compra = json_obj['result']['buy'][0]['Rate']
compra = float(compra)

# Quantidade Inicial
btc = 0.01
buyXDN = btc/venda * (1-0.1/100)
transferXDN = buyXDN - 0.01
sellXDN = transferXDN * compra * (1-0.25/100)
transferBTC = sellXDN - 0.001

gain = (transferBTC-btc)/btc

print(gain * 100)

## Bitfinex (não utilizado pq ainda não tenho conta)

bfx = Bitfinex(None, None)

symbols = bfx.pairs()

bitfinex = {}

pbar = tqdm(range(len(symbols)))
pbar.set_description('Processing')

for j in pbar:
    symbol = symbols[j]

    print(symbol)
    orderbook = bfx.orderbook(symbol)
    try:
        ask = float(orderbook['ask'][0]['price'])
    except:
        ask = 0.0
    try:
        bid = float(orderbook['bid'][0]['price'])
    except:
        bid = 0.0
        
    bitfinex[symbol] = {'ask':ask,'bid':bid, 'base':symbol[-3:]}

## Read from Kraken

In [ ]:
krakenClient = krakenex.API()
krakenClient.load_key('kraken.key')

symbols2 = krakenClient.query_public('AssetPairs')['result']
symbols = list()

for i in symbols2:
    symbols.append(symbols2[i])

kraken = {}

pbar = tqdm(range(len(symbols)))
pbar.set_description('Processing')

for j in pbar:
    symbol = symbols[j]
    
    try:
        ticker = krakenClient.query_public('Ticker', {'pair': symbol['altname']})['result']
        try:
            ask = float(ticker[symbol['altname']]['a'][0])
        except:
            ask = 0.0
        try:
            bid = float(ticker[symbol['altname']]['b'][0])
        except:
            bid = 0.0
            
        # Casos a parte
        if symbol['quote'] == 'XXBT':
            symbol['quote'] = 'BTC'
        elif symbol['quote'] == 'ZEUR':
            symbol['quote'] = 'EUR'
        elif symbol['quote'] == 'XETH':
            symbol['quote'] = 'ETH'
        elif symbol['quote'] == 'ZUSD':
            symbol['quote'] = 'USD'
            
        if symbol['base'] == 'XXRP':
            symbol['base'] = 'XRP'
        elif symbol['base'] == 'XXMR':
            symbol['base'] = 'XMR'
        elif symbol['base'] == 'XXDG':
            symbol['base'] = 'XDG'
        elif symbol['base'] == 'XXBT':
            symbol['base'] = 'BTC'
        elif symbol['base'] == 'XETH':
            symbol['base'] = 'ETH'
        elif symbol['base'] == 'XETC':
            symbol['base'] = 'ETC'
        elif symbol['base'] == 'XLTC':
            symbol['base'] = 'LTC'
        elif symbol['base'] == 'XICN':
            symbol['base'] = 'ICN'
        elif symbol['base'] == 'XMLN':
            symbol['base'] = 'MLN'
        elif symbol['base'] == 'XREP':
            symbol['base'] = 'REP'
        elif symbol['base'] == 'XXLM':
            symbol['base'] = 'XLM'
        elif symbol['base'] == 'XZEC':
            symbol['base'] = 'ZEC'

        kraken[symbol['base']+symbol['quote']] = {'ask':ask,'bid':bid, 'base':symbol['quote'],
                                                  'payinEnabled':True,
                                                  'payoutEnabled':True,
                                                  'takerFee':'0.0016',
                                                  'withdrawalFee': None}
    except:
        ibis = 0
        
exchanges['kraken'] = kraken